In [1]:
import sys
sys.path.append('../')

In [3]:
import generation_utilities
import numpy as np
import librosa
from glob import glob
import random
import IPython.display as ipd
import soundfile as sf
import importlib
import ipywidgets as widgets
import numpy as np
importlib.reload(generation_utilities)
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

unet\diffusion_pytorch_model.safetensors not found
unet\diffusion_pytorch_model.safetensors not found


In [6]:
music = glob("../input_songs/*.mp3")

## Create music dictionary with keys as song names and values as path
music_dict = {}
for song in music:
    song_name = song.split("\\")[-1].split(".")[0]
    music_dict[song_name] = song

music_dict

{'22': '../input_songs\\22.mp3',
 'Anti-Hero': '../input_songs\\Anti-Hero.mp3',
 'Back-To-December': '../input_songs\\Back-To-December.mp3',
 'Blank-Space': '../input_songs\\Blank-Space.mp3',
 'Cardigan': '../input_songs\\Cardigan.mp3',
 'Delicate': '../input_songs\\Delicate.mp3',
 'Love-Story': '../input_songs\\Love-Story.mp3',
 'Lover': '../input_songs\\Lover.mp3',
 'uploaded_song': '../input_songs\\uploaded_song.mp3',
 'Willow': '../input_songs\\Willow.mp3',
 'You-Belong-With-Me': '../input_songs\\You-Belong-With-Me.mp3'}

In [8]:
def load_songs_as_numpy_array(song_dict):
    song_names = list(song_dict.keys())
    song_paths = list(song_dict.values())

    song_selector = widgets.SelectMultiple(
        options=song_names,
        description='Select songs:',
        rows=len(song_names),
        layout=widgets.Layout(width='400px')
    )

    ## Add a slider between 0 and 1 to select the similarity
    similarity_slider = widgets.FloatSlider(
        value=0.75,
        min=0,
        max=1.0,
        step=0.05,
        description='Similarity:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )


    select_button = widgets.Button(description='Select')
    output = widgets.Output()

    def on_select_button_clicked(_):
        selected_songs = [librosa.load(song_paths[i], sr=22050)[0] for i in song_selector.index]      
        name = '+'.join([song_names[i] for i in song_selector.index])

        with output:
            output.clear_output()
            print(f'Selected songs: {name}')
            ## Get similarity from the slider
            similarity = similarity_slider.value
            spec, song = generation_utilities.generate_songs(selected_songs, similarity=similarity, quality=500, merging_quality=100, device='cuda')

            ## Play all the selected songs
            print("Selected songs: ")
            for s in selected_songs:
                ipd.display(ipd.Audio(s, rate=22050))

            # Play song
            print("\n\nGenerated song: ")
            ipd.display(ipd.Audio(song, rate=22050))

            ## Get a rating from the user with ipywidgets
            rating = widgets.Text(
                value='',
                placeholder='Type something',
                description='Rating:',
                disabled=False
            )
            display(rating)
            ## Make a submit button
            submit_button = widgets.Button(description='Submit')
            display(submit_button)

            def on_submit_button_clicked(_):
                ## Convert rating to float
                rating_value = float(rating.value)

                ## Save the song
                total_files = glob('../DataSet/Song/*')
                sf.write(f'../DataSet/Song/{len(total_files)}_{name}_{similarity}_{rating_value}.wav', song, 22050, 'PCM_24')

                ## Save the spectrogram
                total_files = glob('../DataSet/Spec/*')
                np.save(f'../DataSet/Spec/{len(total_files)}_{name}_{similarity}_{rating_value}.npy', spec)

                with output:
                    output.clear_output()
                    print("Saved")


            submit_button.on_click(on_submit_button_clicked)
        
    select_button.on_click(on_select_button_clicked)

    display(song_selector)
    display(select_button)
    display(similarity_slider)
    display(output)


    # return selected_songs, name


In [9]:
load_songs_as_numpy_array(music_dict)

SelectMultiple(description='Select songs:', layout=Layout(width='400px'), options=('22', 'Anti-Hero', 'Back-To…

Button(description='Select', style=ButtonStyle())

FloatSlider(value=0.75, continuous_update=False, description='Similarity:', max=1.0, step=0.05)

Output()

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [5]:
generation_utilities.generate_songs([librosa.load(music_dict["22"], sr=22050)[0]]     ,)

Merging songs...
Generating song...


  0%|          | 0/100 [00:00<?, ?it/s]

(<PIL.Image.Image image mode=L size=256x256>,
 array([-2.02207055e-04, -9.85335573e-05, -1.04858875e-04, ...,
        -2.07161275e-03, -6.43183384e-03, -6.42244611e-03], dtype=float32))

In [14]:
from streamlit_extras.switch_page_button import switch_page

switch_page("New page name")